Test out code for lambda trigger

In [1]:
import boto3
import time

In [2]:
TABLE_NAME = "ngwaf-test"
BUCKET_NAME = "ngwaf-test-bucket-2022-06-13"
NOTEBOOK_NAME = "ngwaf-test-v1"

In [7]:
def trigger(target_s3_file=None):
    db = boto3.client("dynamodb")
    s3 = boto3.client("s3")
    sm = boto3.client("sagemaker")
    
    # Check pipeline status
    pipeline_available = db.get_item(
        TableName=TABLE_NAME,
        Key={"job_key": {"S": "_pipeline_status"}}
    )["Item"]["pipeline_available"]["BOOL"]
    
    if not pipeline_available:
        return {"StatusCode": 400, "Message": "Training job currently in progress"}
    
    # If pipeline available
    new_key = "job-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
#     # Move dataset
#     s3.copy_object(
#         CopySource=target_s3_file,
#         Bucket=BUCKET_NAME,
#         Key="_tmp_train/data.csv"
#     )
    
    # Add new job
    db.put_item(
        TableName="ngwaf-test",
        Item={
            "job_key": {"S": new_key},
            "status": {"S": "training"}
        }
    )
    
    # Update status to busy
    db.update_item(
        TableName=TABLE_NAME,
        Key={"job_key": {"S": "_pipeline_status"}},
        AttributeUpdates={
            "pipeline_available": {"Value": {"BOOL": False}, "Action": "PUT"},
            "pipeline_job_key": {"Value": {"S": new_key}, "Action": "PUT"}
        }
    )
    
    # Start notebook
#     sm.start_notebook_instance(NotebookInstanceName=NOTEBOOK_NAME)
    
    return {"StatusCode": 200, "Message": "Training has kicked off"}

In [8]:
trigger()

{'StatusCode': 200, 'Message': 'Training has kicked off'}